**Name: Rabin BK <br/>
Matriculation number: 23272000**

# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

# Step 1: Load and filter data

In [2]:
gene_exp_df = pd.read_csv('TCGA-BRCA.htseq_fpkm.tsv',
                            sep='\t')
phenotype_df = pd.read_csv('TCGA-BRCA.GDC_phenotype.tsv',
                            sep='\t', index_col=0)

Since we want to map the `gene_exp_df` with sample types in `phenotype_df` frame we transpose the `gene_exp_df` dataframe. In order to do that,
- Firstly we transpose the `gene_exp_df` and reset the index. This will transpose the data frame but will have integers as index and column headers
- So, w re-assign the columns with values in the 0 integer location using `.iloc[]`
- Then drop the 0th axis
- Set `Enembl_ID` as our index 

In [3]:
# transposing the column and reseting the index
gene_exp_df = gene_exp_df.T.reset_index()

# Rename the columns
gene_exp_df.columns = gene_exp_df.iloc[0]
gene_exp_df = gene_exp_df.drop(0)
gene_exp_df = gene_exp_df.set_index('Ensembl_ID')

In [4]:
gene_exp_df.head()

,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,ENSG00000259883.1,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
Ensembl_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-E9-A1NI-01A,0.091708,0.019573,2.235898,0.0,2.321945,3.620056,0.0,0.337087,7.705589,0.084661,...,0.0,0.073008,0.0,0.0,0.0,3.680055,0.28564,0.0,0.599579,0.0
TCGA-A1-A0SP-01A,0.0,0.004701,1.863334,0.0,4.226699,3.546117,0.0,0.016016,6.835508,0.0,...,0.0,0.0,0.0,0.105328,0.055477,3.969785,0.115149,0.0,1.382192,0.0
TCGA-BH-A1EU-11A,0.057899,0.016302,1.704753,0.0,1.975755,3.396943,0.0,0.041455,7.12531,0.461624,...,0.0,0.039503,0.0,0.092108,0.0,3.011921,0.384451,0.0,0.629043,0.0
TCGA-A8-A06X-01A,0.0,0.0,1.947481,0.0,2.808757,4.72327,0.0,0.002361,7.259318,0.088912,...,0.0,0.118749,0.0,0.0,0.0,4.059347,0.345883,0.0,0.396315,0.0
TCGA-E2-A14T-01A,0.0,0.0,2.73469,0.0,1.964479,3.770091,0.0,0.111647,7.643035,0.066036,...,0.0,0.0,0.0,0.113546,0.0,4.249147,0.065679,0.0,0.157504,0.0


We will read the primary tumor samples and healthy tissue samples from the `phenotype_df` 

In [5]:
# getting primary tumor samples and healthy tissue samples

primary_tumor_samples = phenotype_df[phenotype_df['sample_type.samples'] == 'Primary Tumor']
healthy_tissue_samples = phenotype_df[~(phenotype_df['sample_type.samples'] == 'Primary Tumor')]

First we'll find all the common samples in the `primary_tumor_samples` and `gene_exp_df` (both of these DataFrames have sample ID as index).<br>
Then will change that into a list. Then we will filter out all group of rows and columns that matches this list from the `gene_exp_df`

In [6]:
# Get the intersection of indices between primary_tumor_samples.index and gene_exp_df.index
common_samples = set(primary_tumor_samples.index).intersection(gene_exp_df.index)

# list of all common samples
common_samples_list = list(common_samples)

# Filter out the common indices from gene_exp_df
primary_tumor_df = gene_exp_df.loc[common_samples_list]
primary_tumor_df.head()

,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,ENSG00000259883.1,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
Ensembl_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-B6-A0I9-01A,0.029266,0.028433,2.240375,0.0,1.919117,3.162452,0.0,0.060453,7.03727,0.0,...,0.0,0.0,0.0,0.092187,0.227459,4.266755,0.164329,0.0,0.372911,0.0
TCGA-B6-A0WX-01A,0.164188,0.0,1.853127,0.0,2.274642,3.075018,0.0,0.00823,7.337595,0.122737,...,0.0,0.0,0.0,0.158899,0.111508,3.024048,0.530763,0.0,0.677479,0.0
TCGA-A2-A0T5-01A,0.0,0.004673,1.416668,0.0,1.514006,3.831163,0.0,0.282588,7.998239,0.090299,...,0.0,0.067018,0.0,0.053308,0.0,3.689812,0.114482,0.0,0.621468,0.0
TCGA-BH-A0B7-01A,0.097573,0.009308,1.876167,0.0,2.094367,3.834176,0.0,0.172872,6.940677,0.147168,...,0.0,0.007584,0.0,0.0,0.05502,3.767196,0.077141,0.0,0.797911,0.0
TCGA-BH-A0HY-01A,0.0,0.0,1.461369,0.0,2.259182,3.071484,0.0,0.001537,8.203131,0.086843,...,0.0,0.0,0.0,0.051243,0.104149,3.74329,0.266314,0.0,0.384738,0.0


We perform similar steps for healthy tissue samples as well.

In [7]:
# Get the intersection of indices between healthy_tissue_samples.index and gene_exp_df.index
common_tissue_samples = set(healthy_tissue_samples.index).intersection(gene_exp_df.index)

# list of all common tissue samples
common_tissues_list = list(common_tissue_samples)

# Filter out the common indices from gene_exp_df
healthy_tissue_df = gene_exp_df.loc[common_tissues_list]

# since we 
healthy_tissue_df.head()

,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,ENSG00000259883.1,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
Ensembl_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-A7-A0CH-11A,0.063848,0.0,2.079779,0.0,1.720528,3.278227,0.0,0.006183,7.450401,0.604898,...,0.0,0.02196,0.0,0.051617,0.104895,3.529071,0.299686,0.0,0.764413,0.0
TCGA-BH-A203-11A,0.88486,0.004573,1.528933,0.0,2.777692,3.176227,0.0,0.040168,7.393084,0.300738,...,0.0,0.0,0.0,0.0,0.053993,2.666781,0.238304,0.0,0.436469,0.0
TCGA-BH-A18L-11A,0.25448,0.0,1.757252,0.0,2.336778,2.993796,0.0,0.037253,6.824058,0.708796,...,0.0,0.07014,0.0,0.340512,0.0,3.439219,0.203419,0.0,0.702607,0.0
TCGA-BH-A1FD-11B,0.797153,0.008413,1.466558,0.0,1.701458,3.086214,0.0,0.015787,7.310601,0.411371,...,0.0,0.006855,0.0,0.094729,0.0,2.827587,0.15809,0.0,0.476761,0.0
TCGA-BH-A0BS-11A,0.0,0.0,1.513877,0.0,1.725192,3.321137,0.0,0.027152,8.023421,0.382485,...,0.0,0.060143,0.0,0.0,0.109016,3.397684,0.511098,0.0,0.566255,0.0


Since both `primary_tumor_df` and `healthy_tissue_df` are object, it needs to be numeric before passing into <br>`mannwhitneyu()` method. If passed without conversion, then it will produce error. I realized after someone replied to my question in the [Stack Overflow](https://stackoverflow.com/questions/78341239/typeerror-ufunc-isnan-not-supported-for-the-input-types-while-performing-ma?noredirect=1#comment138118974_78341239)

In [8]:
primary_tumor_df = primary_tumor_df.apply(pd.to_numeric, errors='coerce')
healthy_tissue_df = healthy_tissue_df.apply(pd.to_numeric, errors='coerce')

# Step 2: Identify differentially expressed genes

**Finding genes that are differentially expressed**

We perform the Mann Whitney U test to identify differentially expressed genes. Under the general formulation, the test is only consistent when the following occurs under H1: <br>

The probability of an observation from population X exceeding an observation from population Y is different (larger, or smaller) than the probability of an observation from Y exceeding an observation from X; i.e., $P(X > Y) ≠ P(Y > X) or P(X > Y) + 0.5 · P(X = Y) ≠ 0.5$.
<br>
Source: [wikipedia](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test)
<br>
<br>
In our case, X is `primary_tumor_df` and Y is `healthy_tissue_df`.

In [9]:
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [10]:
# Performing Mann-Whitney U test for each gene
# mannwhitneyu is vectorized and works on columns by default so 
# we don't use for loops here
stat, p_value = mannwhitneyu(primary_tumor_df, healthy_tissue_df, axis=0, 
                             alternative='two-sided')

**Multiple testing correction**

Multiple testing corrections adjust p-values derived from multiple statistical tests to correct for occurrence of false positives. Here,  false positives are genes that are found to be statistically different between conditions, but are not in reality. <br>
There are 4 available options (in order of their stringency):
- Bonferroni
- Bonferroni Step-down (Holm)
- Westfall and Young Permutation
- Benjamini and Hochberg False Discovery Rate 

The default multiple testing correction is the **Benjamini and Hochberg False Discovery** Rate. It is the least stringent of all corrections and provides a good balance between discovery of statistically significant genes and limitation of false positive occurrences. We will use `fdr_bh` for Benjamini and Hochberg.

<br> 
Source: https://physiology.med.cornell.edu/people/banfelder/qbio/resources_2008/1.5_GenespringMTC.pdf


In [23]:
# Benjamini-Hochberg procedure for multiple testing correction
adjusted_p_values = multipletests(p_value, method='fdr_bh')[1]

Computing the **log2 fold change** between `primary_tumor_df` and `healthy_tissue_df` samples for each gene

In [29]:
mean_expression_primary_tumor = primary_tumor_df.mean(axis=1)
mean_expression_healthy_tissue = healthy_tissue_df.mean(axis=1)
log2_fold_change = np.log2(mean_expression_primary_tumor / mean_expression_primary_tumor)

In [37]:
# Step 4: Generate DataFrame
results_df = pd.DataFrame({
    'Gene': primary_tumor_df.columns,
    'P-value': p_value,
    'Adjusted P-value': adjusted_p_values,
    'Log2 Fold Change': log2_fold_change
}, index=primary_tumor_df.index)

# Step 5: Write results to CSV
results_df.to_csv('differential_expression_results.csv', index=False)

ValueError: Length of values (60483) does not match length of index (1097)

In [39]:
p_value

array([5.96161718e-49, 8.93986011e-01, 4.16272023e-16, ...,
       6.96114103e-01, 5.68836171e-02, 6.35120556e-01])